Business Challenge: EDA and SQL


In [ ]:
import csv
import chardet

file_path = "Global Health Statistics.csv"

# count numer of rows
with open(file_path, 'r', encoding='utf-8') as file:
    reader = csv.reader(file)
    row_count = sum(1 for row in reader)

# show encoding
with open(file_path, 'rb') as file:
    raw_data = file.read()
    result = chardet.detect(raw_data)
    encoding = result['encoding']

print(f"Total number of rows: {row_count}")
print(f"Detected Encoding: {encoding}")


In [ ]:
import csv
import chardet

# The original dataset is taking too long to insert, so I split into 10 chunks. 

input_file = "Global Health Statistics.csv"
num_chunks = 10  # Number of chunks to split the file into

# Detect encoding of the file
with open(input_file, 'rb') as file:
    raw_data = file.read()
    result = chardet.detect(raw_data)
    encoding = result['encoding']

# Count total rows in the file
with open(input_file, 'r', encoding=encoding) as infile:
    total_rows = sum(1 for _ in infile) - 1  # Subtract 1 for the header row

# Calculate rows per chunk
rows_per_chunk = total_rows // num_chunks
remainder = total_rows % num_chunks  # Handle any leftover rows

# Split the file into chunks
with open(input_file, 'r', encoding=encoding) as infile:
    reader = csv.reader(infile)
    header = next(reader)  # Save the header row

    chunk_number = 1
    rows = []

    for i, row in enumerate(reader, start=1):
        rows.append(row)
        if len(rows) == rows_per_chunk + (1 if remainder > 0 else 0):  # Add 1 row for chunks with remainder
            output_file = f"chunk_{chunk_number}.csv"
            with open(output_file, 'w', encoding='utf-8', newline='') as outfile:
                writer = csv.writer(outfile)
                writer.writerow(header)  # Write the header
                writer.writerows(rows)
            chunk_number += 1
            rows = []  # Clear the list for the next chunk
            if remainder > 0:
                remainder -= 1  # Decrease the remainder for the next chunk

    # Write the remaining rows
    if rows:
        output_file = f"chunk_{chunk_number}.csv"
        with open(output_file, 'w', encoding='utf-8', newline='') as outfile:
            writer = csv.writer(outfile)
            writer.writerow(header)
            writer.writerows(rows)

print(f"File successfully split into {chunk_number} chunks.")


In [ ]:
import os

# Path to the folder containing the chunks
chunk_folder = r"C:\ProgramData\MySQL\MySQL Server 9.1\Uploads\chunks"

# Connect to MySQL

# Load each chunk file sequentially
for chunk_number in range(1, 11):  # Assuming chunks are named chunk_1.csv to chunk_10.csv
    chunk_file = os.path.join(chunk_folder, f"chunk_{chunk_number}.csv")
    print(f"Loading {chunk_file}...")
    
    # Construct the LOAD DATA INFILE query
    query = """
    LOAD DATA INFILE '{chunk_file}'
    INTO TABLE healthstatistics
    FIELDS TERMINATED BY ',' 
    ENCLOSED BY '"'
    LINES TERMINATED BY '\\n'
    IGNORE 1 LINES
    (
        Country, Year, DiseaseName, DiseaseCategory, PrevalenceRate, IncidenceRate, MortalityRate, AgeGroup, Gender, PopulationAffected, 
        HealthcareAccess, DoctorsPer1000, HospitalBedsPer1000, TreatmentType, AverageTreatmentCost, AvailabilityOfVaccinesTreatment, RecoveryRate, DALYs,
        ImprovementIn5Years, PerCapitaIncome, EducationIndex, UrbanizationRate
    );
    """.format(chunk_file=chunk_file.replace("\\", "/"))
    
    try:
        cursor.execute(query)
        connection.commit()
        print(f"Successfully loaded {chunk_file}")
    except pymysql.MySQLError as e:
        print(f"Error loading {chunk_file}: {e}")

# Close the connection
# cursor.close()
# connection.close()
# print("All chunks loaded successfully.")

In [ ]:
# import pymysql

# Switch to SQLAlchemy, because of UserWarning: 
from sqlalchemy import create_engine

# Database connection settings
host = "localhost"  # host, e.g., 127.0.0.1 or a server address
user = "root"  #  MySQL username
password = "Malcomx1"  #  MySQL password
database = "GlobalHealth"  #  database name

# # Establish the connection
# connection = pymysql.connect(
#     host=host,
#     user=user,
#     password=password,
#     database=database
# )

# creates a SQLAlchemy engine, which serves as the entry point for interacting with the database
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{database}")

print(f"Connected to the {database} database successfully!")


In [ ]:
# Switch to SQLAlchemy, because of UserWarning: 

import pandas as pd

query = """
SELECT *
FROM HealthStatistics
WHERE RAND() <= 0.2;
"""

df_healtstatistics_sample = pd.read_sql(query, engine)

print(df_healtstatistics_sample)


In [ ]:
# Switch to SQLAlchemy, because of UserWarning: 

# Trying a filter for Malaria in 2020
query = """
SELECT *
FROM HealthStatistics
WHERE Year = 2020 AND DiseaseName = 'Malaria';
"""
df_filtered = pd.read_sql(query, engine)

print(df_filtered)


In [ ]:
# Question 1:
# Which infectious diseases have the highest prevalence rates globally, and how have these rates changed over the past 5 years?

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# SQL query to fetch data
query = """
SELECT
    DiseaseName,
    DiseaseCategory,
    Year,
    PrevalenceRate
FROM HealthStatistics
WHERE
    DiseaseCategory = 'Infectious'
    AND Year BETWEEN 2020 AND 2024
ORDER BY
    Year DESC;
"""

# Execute the query and load the data into a pandas DataFrame
df_question1 = pd.read_sql(query, engine)

# Analyze the data
# Step 1: Group data to find the highest average prevalence rates globally
highest_prevalence = (
    df_question1.groupby("DiseaseName")
    .agg(
        Avg_Prevalence=("PrevalenceRate", "mean"),
        Total_Prevalence=("PrevalenceRate", "sum")
    )
    .sort_values(by="Avg_Prevalence", ascending=False)
)

# Step 2: Analyze the change in prevalence rates over the last 5 years
recent_years = df_question1[df_question1['Year'] >= df_question1['Year'].max() - 5]

prevalence_change = (
    recent_years.groupby("DiseaseName")
    .agg(
        Initial_Prevalence=("PrevalenceRate", lambda x: x.iloc[0]),
        Latest_Prevalence=("PrevalenceRate", lambda x: x.iloc[-1])
    )
    .assign(Change_in_Prevalence=lambda x: x["Latest_Prevalence"] - x["Initial_Prevalence"])
    .sort_values(by="Change_in_Prevalence", ascending=False)
)

# Step 3: Merge both results
result = highest_prevalence.merge(prevalence_change, on="DiseaseName", how="inner")

# Display results
print("\nTop Infectious Diseases by Prevalence:")
print(highest_prevalence.head(10))

print("\nChange in Prevalence Rates Over the Last 5 Years:")
print(prevalence_change.head(10))

# Step 4: Sort data by year
df_question1_sorted = df_question1.sort_values(by='Year')

# Step 5: Select top 5 diseases by highest average prevalence rates
top_diseases = highest_prevalence.head(5).index

# Filter data for the top 5 diseases
top_diseases_data = df_question1_sorted[df_question1_sorted['DiseaseName'].isin(top_diseases)]

# Aggregate data by DiseaseName and Year
aggregated_data = (
    top_diseases_data.groupby(['DiseaseName', 'Year'])
    .agg({'PrevalenceRate': 'mean'})
    .reset_index()
)

# Plot the aggregated data
sns.set_theme(style="whitegrid")
plt.figure(figsize=(12, 7))
palette = sns.color_palette("husl", len(top_diseases))  # colorful theme

for i, (disease, group) in enumerate(aggregated_data.groupby('DiseaseName')):
    plt.plot(group['Year'], group['PrevalenceRate'], 
             marker='o', label=disease, color=palette[i], linewidth=2.5)

# Add labels and title
plt.title('Prevalence Rate per Year for Top 5 Diseases', fontsize=16, weight='bold')
plt.xlabel('Year', fontsize=14)
plt.ylabel('Prevalence Rate', fontsize=14)
plt.xticks(sorted(aggregated_data['Year'].unique()), fontsize=12, rotation=45)
plt.yticks(fontsize=12)
plt.legend(title="Disease Name", fontsize=12, title_fontsize=14)
plt.grid(True, which='major', linestyle='--', linewidth=0.5, alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# Question 2:
# Which age groups and genders are most affected by high-prevalence infectious diseases? 
# Are there significant disparities?

def get_high_prevalence_data(connection: pymysql.connections.Connection, diseasecategory: str) -> pd.DataFrame:
    """    
    This function queries the `HealthStatistics` table to compute the average prevalence rate and 
    the total affected population for each combination of age group and gender within the specified 
    disease category. It filters the data to include only those entries where the prevalence rate 
    is greater than the average prevalence rate.

    Args:
        connection (pymysql.connections.Connection): A valid database connection created with PyMySQL.
        diseasecategory (str): The disease category to filter on (e.g., 'Infectious').

    Returns:
        pd.DataFrame: A pandas DataFrame containing columns:
                      - `AgeGroup`: The age group.
                      - `Gender`: The gender (Male, Female, Other).
                      - `AvgPrevalence`: The average prevalence rate for the group.
                      - `TotalAffected`: The total population affected for the group.
    """
        
    
    query = """
    SELECT
        AgeGroup,
        Gender,
        AVG(PrevalenceRate) AS AvgPrevalence,
        SUM(PopulationAffected) AS TotalAffected
    FROM HealthStatistics
    WHERE
        DiseaseCategory = %s
        AND PrevalenceRate > (
            SELECT AVG(PrevalenceRate)
            FROM HealthStatistics
            WHERE DiseaseCategory = %s
        )
    GROUP BY AgeGroup, Gender
    ORDER BY AvgPrevalence DESC;
    """
    params = (diseasecategory, diseasecategory)
    return pd.read_sql(query, connection, params=params)

# Call the function
df_question2 = get_high_prevalence_data(connection, 'Infectious')

# df_question2 = pd.read_sql(query_2, connection)

age_order = ['0-18', '19-35', '36-60', '61+']

df_question2['AgeGroup'] = pd.Categorical(df_question2['AgeGroup'], categories=age_order, ordered=True)

# Plot the graph with the updated order
plt.figure(figsize=(10, 6))
sns.barplot(
    data=df_question2,
    x="AgeGroup",
    y="AvgPrevalence",
    hue="Gender",
    palette="viridis"
)
plt.title("Prevalence Rate of by Age Group and Gender")
plt.xlabel("Age Group")
plt.ylabel("Average Prevalence Rate (%)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Question 3: 
# Is there a correlation between healthcare access, the number of doctors per 1000 people, 
# and the recovery rate for specific diseases?

query_3 = """
SELECT
    DiseaseName,
    AVG(HealthcareAccess) AS AvgHealthcareAccess,
    AVG(DoctorsPer1000) AS AvgDoctorsPer1000,
    AVG(RecoveryRate) AS AvgRecoveryRate
FROM HealthStatistics
GROUP BY DiseaseName;
"""

df_question3 = pd.read_sql(query_3, connection)
print(df_question3.head())

correlation_matrix = df_question3[[
    'AvgHealthcareAccess',
    'AvgDoctorsPer1000',
    'AvgRecoveryRate'
]].corr()

print(correlation_matrix)

plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Between Healthcare Access, Doctors, and Recovery Rate")
plt.show()


sns.pairplot(
    df_question3,
    vars=['AvgHealthcareAccess', 'AvgDoctorsPer1000', 'AvgRecoveryRate'],
    kind="reg"
)
plt.suptitle("Relationships Between Healthcare Access, Doctors, and Recovery Rate", y=1.02)
plt.show()



In [ ]:
# Question 4: 
# Which infectious diseases have the highest mortality rates, and how do socioeconomic factors 
# (e.g., per capita income, education index) influence these rates?

query_4 = """
SELECT
    DiseaseName,
    AVG(MortalityRate) AS AvgMortalityRate,
    AVG(PerCapitaIncome) AS AvgPerCapitaIncome,
    AVG(EducationIndex) AS AvgEducationIndex
FROM HealthStatistics
GROUP BY DiseaseName
ORDER BY AvgMortalityRate DESC
LIMIT 10;
"""

df_question4 = pd.read_sql(query_4, connection)
# df_question4.describe()
print(df_question4.head())


# Calculate correlations
correlation_matrix = df_question4[[
    'AvgMortalityRate', 
    'AvgPerCapitaIncome', 
    'AvgEducationIndex'
]].corr()

print(correlation_matrix)

plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Between Mortality Rate and Socioeconomic Factors")
plt.show()

In [ ]:
# Question 5:
# Does the urbanization rate affect the incidence and prevalence rates of infectious diseases? 
# Are urban areas more vulnerable to certain outbreaks?

query = """
SELECT
    DiseaseName,
    AVG(UrbanizationRate) AS AvgUrbanizationRate,
    AVG(IncidenceRate) AS AvgIncidenceRate,
    AVG(PrevalenceRate) AS AvgPrevalenceRate
FROM HealthStatistics
WHERE DiseaseCategory = 'Infectious'
GROUP BY DiseaseName
ORDER BY AvgUrbanizationRate DESC;
"""

# Execute the query and load the data into a pandas DataFrame
df_urbanization = pd.read_sql(query, connection)

df_urbanization.describe()
# Calculate correlation
correlation_matrix = df_urbanization[[
    'AvgUrbanizationRate',
    'AvgIncidenceRate',
    'AvgPrevalenceRate'
]].corr()

print(correlation_matrix)

 
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Between Urbanization and Disease Metrics")
plt.show()



In [ ]:
# 6.	Economic Burden:
# What is the average treatment cost (USD) for the most common infectious diseases, 
# and how does it compare to the per capita income in different countries?

# Define a query to calculate the average treatment cost for common infectious diseases
query_treatment_cost = """
SELECT
    DiseaseName,
    AVG(TreatmentCostUSD) AS AvgTreatmentCost,
    Country,
    AVG(PerCapitaIncome) AS AvgPerCapitaIncome
FROM HealthStatistics
WHERE DiseaseCategory = 'Infectious'
GROUP BY DiseaseName, Country
ORDER BY AvgTreatmentCost DESC;
"""

# Execute the query and load the data into a DataFrame
df_treatment_cost = pd.read_sql(query_treatment_cost, connection)

# Display the first few rows of the resulting DataFrame
df_treatment_cost.head()

# Visualizing the comparison between treatment cost and per capita income


# Create a scatter plot to compare treatment costs and per capita income
plt.figure(figsize=(12, 8))
sns.scatterplot(
    data=df_treatment_cost,
    x="AvgPerCapitaIncome",
    y="AvgTreatmentCost",
    hue="DiseaseName",
    palette="Set2",
    s=100,
    alpha=0.8
)

plt.title("Comparison of Average Treatment Cost and Per Capita Income", fontsize=16)
plt.xlabel("Average Per Capita Income (USD)", fontsize=12)
plt.ylabel("Average Treatment Cost (USD)", fontsize=12)
plt.legend(title="Disease Name", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# Summary statistics of treatment cost vs. per capita income
df_treatment_cost.describe()






In [ ]:
# How does the availability of vaccines or treatments impact the mortality and recovery rates for specific infectious diseases?


query = """
SELECT
    DiseaseName,
    AvailabilityOfVaccinesTreatment,
    AVG(MortalityRate) AS AvgMortalityRate,
    AVG(RecoveryRate) AS AvgRecoveryRate
FROM HealthStatistics
WHERE DiseaseCategory = 'Infectious'
GROUP BY DiseaseName, AvailabilityOfVaccinesTreatment
ORDER BY AvgMortalityRate DESC;
"""

df_vaccine = pd.read_sql(query, connection)
print(df_vaccine.head())

df_summary = df_vaccine.groupby("AvailabilityOfVaccinesTreatment").agg({
    "AvgMortalityRate": "mean",
    "AvgRecoveryRate": "mean"
}).reset_index()

print(df_summary)


# Bar plot for Mortality and Recovery Rates by Vaccine Availability
df_melted = df_summary.melt(id_vars="AvailabilityOfVaccinesTreatment", 
                            value_vars=["AvgMortalityRate", "AvgRecoveryRate"],
                            var_name="Metric", 
                            value_name="Rate")

plt.figure(figsize=(8, 5))
sns.barplot(data=df_melted, x="AvailabilityOfVaccinesTreatment", y="Rate", hue="Metric", palette="viridis")
plt.title("Impact of Vaccine Availability on Mortality and Recovery Rates")
plt.xlabel("Availability of Vaccines/Treatments")
plt.ylabel("Rate (%)")
plt.tight_layout()
plt.show()



In [ ]:
# Which diseases contribute the most to DALYs (Disability-Adjusted Life Years)?


query = """
SELECT
    DiseaseName,
    SUM(DALYs) AS TotalDALYs
FROM HealthStatistics
GROUP BY DiseaseName
ORDER BY TotalDALYs DESC
LIMIT 10;
"""

# Execute the query and load the data into a DataFrame
df_dalys = pd.read_sql(query, connection)

# Display the top diseases contributing to DALYs
print(df_dalys)


# Bar plot for top diseases by DALYs
plt.figure(figsize=(10, 6))
sns.barplot(data=df_dalys, x="TotalDALYs", y="DiseaseName", palette="viridis")
plt.title("Top Diseases Contributing to DALYs Globally")
plt.xlabel("Total DALYs")
plt.ylabel("Disease Name")
plt.tight_layout()
plt.show()
